In [100]:
import pandas as pd
import numpy as np

In [134]:
faostat_area_harvested = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/faostat_area_harvested_finall.csv")
gfrac = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC Area HarvestedRegionType_fulll.csv")
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")

In [135]:
faostat_area_harvested = faostat_area_harvested.rename(columns={"Value": "Fao Area Harvested", "IMAGE Classification": "NGFBFC", "Year": "time"})
faostat_area_harvested_merge = pd.merge(left=faostat_area_harvested, right=country_code, left_on='Area', right_on='ISO Country')
faostat_area_harvested_columns = faostat_area_harvested_merge[['IMAGE Region Name', 'time', 'NGFBFC', 'Fao Area Harvested']]
faostat_area_harvested_columns = faostat_area_harvested_columns.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
faostat_area_harvested_index = faostat_area_harvested_columns.reset_index()
faostat_area_harvested_index.head()

,IMAGE Region Name,time,NGFBFC,Fao Area Harvested
0,Brazil,1970,Maize,10037136.6
1,Brazil,1970,Oil & palm fruit,3560.0
2,Brazil,1970,Other non-food & luxury & spices,3171196.2
3,Brazil,1970,Other temperate cereals,90177.6
4,Brazil,1970,Plant based fibres,4588723.2


In [136]:
faostat_area_harvested_index[(faostat_area_harvested_index['IMAGE Region Name'] == "China") & (faostat_area_harvested_index['NGFBFC'] == 'Maize')]

,IMAGE Region Name,time,NGFBFC,Fao Area Harvested
462,China,1970,Maize,16264540.6
478,China,1975,Maize,18313912.6
494,China,1980,Maize,19704137.2
510,China,1985,Maize,18940453.2
526,China,1990,Maize,20891783.2
542,China,1995,Maize,22651361.2
558,China,2000,Maize,24655507.8
574,China,2005,Maize,26783793.0
590,China,2010,Maize,32442078.4
606,China,2015,Maize,41024654.6


In [137]:
gfrac = gfrac.rename(columns={"value": "GFRAC"})
gfrac.head()

,IMAGE Region Name,NGFBFC,time,GFRAC
0,Brazil,Maize,1970,1.129294e+07
1,Brazil,Maize,1975,1.208552e+07
2,Brazil,Maize,1980,1.078206e+07
3,Brazil,Maize,1985,1.065404e+07
4,Brazil,Maize,1990,1.301961e+07


In [138]:
gfrac_copy = gfrac.copy()
gfrac_copy.loc[:,'FAO'] = 0.0
gfrac_copy['FAO'] = gfrac.groupby(['NGFBFC', 'time', 'IMAGE Region Name'])['GFRAC'].transform(lambda x: faostat_area_harvested_index[(faostat_area_harvested_index['NGFBFC'] == x.name[0]) & (faostat_area_harvested_index['time'] == x.name[1]) & (faostat_area_harvested_index['IMAGE Region Name'] == x.name[2])]['Fao Area Harvested'].sum())

In [140]:
gfrac_copy.loc[:, 'selisih'] = abs(gfrac_copy['GFRAC'] - gfrac_copy['FAO'])

In [141]:
gfrac_copy[(gfrac_copy['IMAGE Region Name'] == "China") & (gfrac_copy['NGFBFC'] == 'Maize')]

,IMAGE Region Name,NGFBFC,time,GFRAC,FAO,selisih
528,China,Maize,1970,1.638164e+07,16264540.6,1.170949e+05
529,China,Maize,1975,1.860431e+07,18313912.6,2.903943e+05
530,China,Maize,1980,1.991471e+07,19704137.2,2.105719e+05
531,China,Maize,1985,1.944587e+07,18940453.2,5.054188e+05
532,China,Maize,1990,2.123221e+07,20891783.2,3.404304e+05
533,China,Maize,1995,2.324823e+07,22651361.2,5.968721e+05
534,China,Maize,2000,2.460665e+07,24655507.8,4.885515e+04
535,China,Maize,2005,2.475859e+07,26783793.0,2.025200e+06
536,China,Maize,2010,2.906188e+07,32442078.4,3.380197e+06
537,China,Maize,2015,3.739178e+07,41024654.6,3.632878e+06


In [145]:
gfrac_copy.sort_values(by='selisih', ascending=False)

,IMAGE Region Name,NGFBFC,time,GFRAC,FAO,selisih
3509,Russia,Wheat,1970,3.377263e+07,64043603.2,3.027097e+07
3510,Russia,Wheat,1975,3.229508e+07,61269203.2,2.897413e+07
3511,Russia,Wheat,1980,3.142329e+07,59698400.0,2.827511e+07
3512,Russia,Wheat,1985,2.605690e+07,49512204.8,2.345531e+07
1574,Kazakhstan region,Wheat,1975,1.938951e+07,0.0,1.938951e+07
...,...,...,...,...,...,...
4377,Ukraine region,Tropical roots & tubers,2020,0.000000e+00,0.0,0.000000e+00
4376,Ukraine region,Tropical roots & tubers,2015,0.000000e+00,0.0,0.000000e+00
4375,Ukraine region,Tropical roots & tubers,2010,0.000000e+00,0.0,0.000000e+00
4374,Ukraine region,Tropical roots & tubers,2005,0.000000e+00,0.0,0.000000e+00


In [33]:
gfrac_copy.to_excel("", index=False)